# Window Safety

This notebook document demonstrates an error that often boggles Quantopian beginners, namely, the [NonWindowSafeInput](https://github.com/quantopian/zipline/blob/bba810078c56bf6f2dad9f1e9653f6833ad2fe4e/zipline/errors.py#L517). This error is a bit confusing because there are multiple conditions that triggers this error. 

NonWindowSafe error shows up in these scenarios:

**Scenario 1: Adding `latest` to longer than 1-day window**

Condition: Using any data with `latest` into a Factor that uses >1 `window_length`.

Fix: To fix this, well, remove the `latest` part.

**Scenario 2: Using moving average factor in another factor that is longer than 1-day window**

Since we are working with historical data, it is possible that prices get adjusted due to stock splits, dividends, etc. Factors that take these into account e.g. any moving average factor cannot be passed into another factor-creation function. Other functions, like return-related functions, can be passed without triggering this error.

Therefore, the condition for this error is as follows:

1. Moving average factor is used as an input in another factor.
2. `window_length` is set to `>1`.

In [1]:
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.morningstar import Fundamentals
from quantopian.pipeline.factors import CustomFactor, Returns, BollingerBands
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline

## Scenario 1 Demonstration

In this scenario, both `returns` and `signal` would cause `NonWindowSafeInput` error. Removal of the `latest` attribute fixes this code.

In [2]:
# Uses USEquityPricing.close.latest for window_length > 1

class TestFactor(CustomFactor):
    inputs=[]
    def compute(self, today, assets, out, input1):
        out[:] = input1[-1]


try:
    returns = Returns(inputs=[USEquityPricing.close.latest], window_length=2)
    signal = TestFactor(inputs=[USEquityPricing.close.latest], window_length=2)
    
except Exception as e:
    if 'NonWindowSafeInput' in str(type(e)):
        print("Caught NonWindowSafeInput")
    else:
        raise

Caught NonWindowSafeInput


## Scenario 2 Demonstration

BollingerBands algorithm uses moving average for calculating the bands, and hence, its results cannot be passed into another factor that is longer than 1 day.

For the code below, `signal1` does not return an error, but `signal2` does. It is also okay to pass if you changed the `window_length` to 1.

In [3]:
class TestFactor(CustomFactor):
    inputs=[]
    def compute(self, today, assets, out, input1):
        out[:] = input1[-1]


try:
    returns = Returns(inputs=[USEquityPricing.close], window_length=22)
    bb1, bb2, bb3 = BollingerBands(
            inputs=[USEquityPricing.close],
            window_length=22,
            k=2)
    
    signal1 = TestFactor(inputs=[returns], window_length=22)
    signal2 = TestFactor(inputs=[bb1], window_length=22)

except Exception as e:
    if 'NonWindowSafeInput' in str(type(e)):
        print("Caught NonWindowSafeInput")
    else:
        raise

Caught NonWindowSafeInput
